# Medium Blog Summarizer - With Ollama using OpenAI API Framework

In [ ]:
!ollama pull llama3.2

In [ ]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [ ]:
# A class to represent a Webpage
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
system_prompt_guru = "You are a scientific assistant. You have to help summarize a blog post.\
Please tell about the most important findings and anything the author emphasizes.\
Give the output in markdown."


In [ ]:
guru_blog = "https://gurudeep1998.medium.com/w-net-a-deep-model-for-fully-unsupervised-image-segmentation-reproduction-2651540eaed6"



In [ ]:
# Ollama runs a server where requests can be sent for computation
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
def user_prompt_for_blog(website):
    user_prompt = f"You are looking at a blog titled {website.title}"
    user_prompt += "\nThe contents of this blog is as follows; \
please provide a summary of this blog in markdown. \
If it includes references any research papers and github links, please provide their links also.\n\n"
    user_prompt += website.text
    return user_prompt


def summarize_medium_blog_ollama(url, system_prompt, model="llama3.2"):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = model,
        messages = [{"role": "system", "content": system_prompt},
           {"role": "user", "content": user_prompt_for_blog(website)}]
    )
    return response.choices[0].message.content

In [ ]:

print(display(Markdown(summarize_medium_blog_ollama(guru_blog, system_prompt_guru, "llama3.2"))))

In [ ]:
# Let's check with another model
!ollama pull deepseek-r1:1.5b

In [ ]:
print(display(Markdown(summarize_medium_blog_ollama(guru_blog, system_prompt_guru, "deepseek-r1:1.5b"))))